In [1]:
import numpy as np
import scipy.io as sio
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [1]:
class NKM:
    def __init__(self, β=0.99, ε=6.0, θ=2/3, σ=1, φ=1, α=1/3, φ_pi=1.5, 
                 φ_y=0.5/4, η=4, ρ_v=0.5, ρ_a=0.9, ρ_z=0.5, 
                 taylor ='i = rho+phi_pi*pi+phi_y*y_gap+v;',
                 vcov=[[0.0625, 0, 0], [0, 1, 0], [0 , 0, 0.0625]]):

        self.β, self.ε, self.θ, self.σ, self.φ, self.α, self.φ_pi, self.φ_y, self.η, self.ρ_v, self.ρ_a, self.ρ_z = β, ε, θ, σ, φ, α, φ_pi, φ_y, η, ρ_v, ρ_a, ρ_z
        self.ρ = 1/β-1
        self.Ψ_yan=(1+φ)/(σ*(1-α)+φ+α)
        self.Θ=(1-α)/(1-α+α*ε)
        self.λ = (1-self.θ)*(1-β*self.θ)*self.Θ/self.θ
        self.κ = self.λ*(σ+(φ+α)/(1-α))
        self.μ=np.log(ε/(ε-1))
        self.θ_yn=-((1-α)*(self.μ-np.log(1-α)))/(σ*(1-α)+φ+α)

        self.variables, self.varexo, self.model = self.dict_model()
        self.model['i']=taylor
        self.parameters=self.dict_parameters()
        self.vcov = vcov


    #1.Parameters definition    
    def dict_parameters(self):
        β, ε, θ, σ, φ, α, φ_pi, φ_y, η, ρ_v, ρ_a, ρ_z = self.β, self.ε, self.θ, self.σ, self.φ, self.α, self.φ_pi, self.φ_y, self.η, self.ρ_v, self.ρ_a, self.ρ_z
        ρ, Ψ_yan, Θ, λ, κ, μ, θ_yn = self.ρ, self.Ψ_yan, self.Θ, self.λ, self.κ, self.μ, self.θ_yn
        parameters={'beta':β, 'epsilon':ε, 'theta':θ, 'sigma':σ, 
                    'phi':φ, 'alpha':α, 'phi_pi':φ_pi, 'phi_y':φ_y, 
                    'eta':η, 'rho_v':ρ_v, 'rho_a':ρ_a, 'rho_z':ρ_z, 
                    'rho':ρ, 'PSI_yan':Ψ_yan, 'THETA':Θ, 'lambda':λ, 
                    'kappa':κ, 'mu':μ, 'vartheta_yn':θ_yn }
        
        return parameters

    #2.Model definition    
    def dict_model(self):

        #VARIABLES
        variables=['pi', 'y_gap', 'y', 'rn', 'i', 'n', 'a', 'v', 'm_growth', 'm_r', 'r_real', 'y_nat', 'i_ann', 'r_ann', 'pi_ann', 'c', 'l', 'z']
        
        #VAREXO
        varexo= ['eps_v', 'eps_a', 'eps_z']
        
        #MODEL
        nkpc ='pi = beta*pi(+1)+kappa*y_gap+z;' #// (21) +z (ar1) 
        dis ='y_gap = y_gap(+1)-1/sigma*(i-pi(+1)-rn);' #y_gap is output gap (22) 
        taylor ='i = rho+phi_pi*pi+phi_y*y_gap+v;'
        rn = 'rn=rho+sigma*PSI_yan*(a(+1)-a); '
        r_real ='r_real=i-pi(+1); '
        y_nat = 'y_nat= PSI_yan*a + vartheta_yn; '
        y_gap = 'y_gap=y-y_nat;'
        v = 'v = rho_v*v(-1) + eps_v; '
        a = 'a = rho_a*a(-1) + eps_a;'
        z = 'z = rho_z*z(-1) + eps_z; '
        y = 'y = a+(1-alpha)*n;'
        m_growth = 'm_growth = 4*(y-y(-1)-eta*(i-i(-1))+pi);'
        m_r = 'm_r = y-eta*i;'
        c = 'c=y;'
        l = 'l=1-n;'
        i_ann = 'i_ann=4*i;'
        r_ann = 'r_ann=4*r_real;'
        pi_ann = 'pi_ann=4*pi;'
        
        

        model = {'pi': nkpc, 'y_gap':dis, 'i':taylor, 'rn':rn, 'r_real':r_real, 'y_nat':y_nat, 
                'y_gap2':y_gap, 'v':v, 'a':a, 'z':z, 'y':y, 'm_growth':m_growth, 'm_r':m_r,
                'c':c, 'l':l, 'i_ann':i_ann, 'r_ann':r_ann, 'pi_ann':pi_ann}
        
        return variables, varexo, model

    #3.Create .mod file
    def mod_file(self):
        #Preamble
        variables = self.variables
        varexo = self.varexo
        parameters = self.parameters
        model = self.model
        vcov = self.vcov
        
#         parameters=dict_parameters()
        preamble=['// The basic New Keynesian model\n', '// ----------------------------------------\n']
               
        #Modify .mod file
        nkm_mod=open('Gali_new.mod', 'w')
        #string_list = nkm_mod.readlines()
        nkm_mod.write(preamble[0]+preamble[1])
        
        #Variables
#         nkm_mod.write('var ')
        nkm_mod.write(" ".join(['var ']+variables+['; \n']))
#         for x in variables:
#             nkm_mod.write(x + ' ')
#         nkm_mod.write('; \n')
        
        #Varexo
#         nkm_mod.write('varexo ')
        nkm_mod.write(" ".join(['varexo ']+varexo+['; \n']))
#         for x in varexo:
#             nkm_mod.write(x + ' ')
#         nkm_mod.write('; \n')
        
        #Parameters
        nkm_mod.write('\n' + '//Parameters \n' + '\n') 
        
#         nkm_mod.write('parameters ' )
        nkm_mod.write(" ".join(['parameters ']+list(parameters.keys())+['; \n']))
#         for x in parameters:
#             nkm_mod.write(x + ' ')
#         nkm_mod.write(' ; \n')
        
#         nkm_mod.write(param[0])
        nkm_mod.write('// ----------------------------------------\n')
        
        for x in parameters:
            nkm_mod.write(x + ' = ' + str(parameters[x]) + ';\n')
        
        #Model
        nkm_mod.write('\n' + '//Model \n' + '\n' + '\n' + 'model;' + '\n' )
#         nkm_mod.write('\n' + 'model;' + '\n')        
        for x in model:
            nkm_mod.write(model[x] + ' \n')
        nkm_mod.write('\n' + 'end; \n')
#         for x in variables

        
        #Initval
        nkm_mod.write('\n' + '//Initial Values \n' + '\n')
        nkm_mod.write('\n' + 'initval;' + '\n')
        
        for x in variables:
            if x == 'n' or x == 'l':
                nkm_mod.write(x + ' = 0.5 ; \n')
#             elif x == 'p':
#                 nkm_mod.write(x + ' = pi ; \n')
            else:
                nkm_mod.write(x + ' = 0 ; \n')
        nkm_mod.write('\n' + 'end;' + '\n')        

        
        #Shock
        nkm_mod.write('\n' + '//Shock \n' + '\n')
        nkm_mod.write('vcov = [')
        for x in range(0,len(vcov)):
            if x == len(vcov)-1:
                for i in vcov[x]:
                    nkm_mod.write(str(i) + ' ')
                nkm_mod.write('];\n')
            else :
                for i in vcov[x]:
                    nkm_mod.write(str(i) + ' ')
                nkm_mod.write(';\n')
        
        nkm_mod.close()
    
    #4. Run model    
    def run_model(self):
        self.mod_file()
        !dynare++ --order=1 --no-centralize Gali_new.mod
        df = sio.loadmat('Gali_new.mat')
        #list_df.append(df)
        return df
        
     
    #5. Keys for plot
    def key_variables(self, df):
        #Variables keys
        df['dyn_vars']
        dict_vars = { i : df['dyn_vars'][i] for i in range(0, len(df['dyn_vars'])) }
        dict_vars

        rn       =list(dict_vars.keys())[list(dict_vars.values()).index('rn      ')]
        n        =list(dict_vars.keys())[list(dict_vars.values()).index('n       ')]
        m_growth =list(dict_vars.keys())[list(dict_vars.values()).index('m_growth')]
        m_r      =list(dict_vars.keys())[list(dict_vars.values()).index('m_r     ')]
        r_real   =list(dict_vars.keys())[list(dict_vars.values()).index('r_real  ')]
        y_nat    =list(dict_vars.keys())[list(dict_vars.values()).index('y_nat   ')]
        i_ann    =list(dict_vars.keys())[list(dict_vars.values()).index('i_ann   ')]
        r_ann    =list(dict_vars.keys())[list(dict_vars.values()).index('r_ann   ')]
        pi_ann   =list(dict_vars.keys())[list(dict_vars.values()).index('pi_ann  ')]
        c        =list(dict_vars.keys())[list(dict_vars.values()).index('c       ')]
        l        =list(dict_vars.keys())[list(dict_vars.values()).index('l       ')]
        y        =list(dict_vars.keys())[list(dict_vars.values()).index('y       ')]
        i        =list(dict_vars.keys())[list(dict_vars.values()).index('i       ')]
        v        =list(dict_vars.keys())[list(dict_vars.values()).index('v       ')]
        z        =list(dict_vars.keys())[list(dict_vars.values()).index('z       ')]
        a        =list(dict_vars.keys())[list(dict_vars.values()).index('a       ')]
        pi       =list(dict_vars.keys())[list(dict_vars.values()).index('pi      ')]
        y_gap    =list(dict_vars.keys())[list(dict_vars.values()).index('y_gap   ')]

        return rn, n, m_growth, m_r, r_real, y_nat, i_ann, r_ann, pi_ann, c, l ,y, i, v, z, a, pi, y_gap     
    
    #6. Plot IRF
    def plot_irf(self, df, shock):
        rn, n, m_growth, m_r, r_real, y_nat, i_ann, r_ann, pi_ann, c, l ,y, i, v, z, a, pi, y_gap=self.key_variables(df[0])
        if shock == 'monetary':
            eps = 'dyn_irfp_eps_v_mean'
            shock_name='v'
        elif shock == 'technology':
            eps = 'dyn_irfp_eps_a_mean'
            shock_name='a'
        elif shock == 'phillips':
            eps = 'dyn_irfp_eps_z_mean'
            shock_name='z'
        else : 
            print('Try another shock: monetary, technology or phillips.')

        x = np.arange(15)
        var=["Output Gap", "Inflation", "Output=Consumption", "Employment", "Natural output", "Leisure", "Nominal Rate", "Real Rate", "Money Growth", shock_name]
        colors = px.colors.sequential.RdBu + px.colors.sequential.thermal + px.colors.sequential.Pinkyl
        fig = make_subplots(rows=5, cols=2, subplot_titles=var)

        for i in range(0,len(df)):
            #Output gap
            fig.add_trace(go.Scatter(x=x, y=df[i][eps][y_gap,0:15], line=dict(color=colors[i], width=2), showlegend=False ), row=1, col=1)
            #Inflation
            fig.add_trace(go.Scatter(x=x, y=df[i][eps][pi_ann,0:15], line=dict(color=colors[i], width=2), showlegend=False), row=1, col=2)
            #Output = consume
            fig.add_trace(go.Scatter(x=x, y=df[i][eps][y,0:15], line=dict(color=colors[i], width=2), showlegend=False), row=2, col=1)
            #Employment
            fig.add_trace(go.Scatter(x=x, y=df[i][eps][n,0:15], line=dict(color=colors[i], width=2), showlegend=False), row=2, col=2)
            #Natural output
            fig.add_trace(go.Scatter(x=x, y=df[i][eps][y_nat,0:15], line=dict(color=colors[i], width=2), showlegend=False), row=3, col=1)
            #Leisure
            fig.add_trace(go.Scatter(x=x, y=df[i][eps][l,0:15], line=dict(color=colors[i], width=2), showlegend=False), row=3, col=2)
            #Nominal rate
            fig.add_trace(go.Scatter(x=x, y=df[i][eps][i_ann,0:15], line=dict(color=colors[i], width=2), showlegend=False), row=4, col=1)
            #Real rate
            fig.add_trace(go.Scatter(x=x, y=df[i][eps][r_ann,0:15], line=dict(color=colors[i], width=2), showlegend=False), row=4, col=2)
            #Money growth
            fig.add_trace(go.Scatter(x=x, y=df[i][eps][m_growth,0:15], line=dict(color=colors[i], width=2), showlegend=False), row=5, col=1)
            #shock
            if shock == 'monetary':
                fig.add_trace(go.Scatter(x=x, y=df[i][eps][v,0:15], line=dict(color=colors[i], width=2), showlegend=False), row=5, col=2)
            elif shock == 'technology':
                fig.add_trace(go.Scatter(x=x, y=df[i][eps][a,0:15], line=dict(color=colors[i], width=2), showlegend=False), row=5, col=2)
            elif shock == 'phillips':
                fig.add_trace(go.Scatter(x=x, y=df[i][eps][z,0:15], line=dict(color=colors[i], width=2), showlegend=False), row=5, col=2)            
            else: 
                print('Try another shock: monetary, technology or phillips.')


        #Title
        if shock == 'monetary':
            fig.update_layout(title_text="Effects of Monetary Policy Shock", height=850)  
        elif shock == 'technology':
            fig.update_layout(title_text="Effects of Technology Shock", height=850)  
        elif shock == 'phillips':
            fig.update_layout(title_text="Effects of Phillips Curve Shock", height=850)    
        else: 
            print('Try another shock: monetary, technology or phillips.')    
         
         # Update xaxis properties
        fig.update_xaxes(dict(dtick = 2), range=[-0.5, 15], row=1, col=1)
        fig.update_xaxes(dict(dtick = 2), range=[-0.5, 15], row=1, col=2)
        fig.update_xaxes(dict(dtick = 2), range=[-0.5, 15], row=2, col=1)
        fig.update_xaxes(dict(dtick = 2), range=[-0.5, 15], row=2, col=2)
        fig.update_xaxes(dict(dtick = 2), range=[-0.5, 15], row=3, col=1)
        fig.update_xaxes(dict(dtick = 2), range=[-0.5, 15], row=3, col=2)
        fig.update_xaxes(dict(dtick = 2), range=[-0.5, 15], row=4, col=1)
        fig.update_xaxes(dict(dtick = 2), range=[-0.5, 15], row=4, col=2)
        fig.update_xaxes(dict(dtick = 2), range=[-0.5, 15], row=5, col=1)
        fig.update_xaxes(dict(dtick = 2), range=[-0.5, 15], row=5, col=2)        
        
        fig.show()

    #7. Matrix of variance and covariance
    def table_vcov(self, df):
        values = df['dyn_vcov']
        values = values.tolist()
        values.insert(0, df['dyn_vars'].tolist())

#         print(values)
        fig = go.Figure(data=[go.Table(
                        columnwidth = [120,100],
                        header=dict(values=['var']+[x for x in df['dyn_vars']],
                                   line_color='darkslategray',
                                    fill_color='royalblue',
                                    font=dict(color='white', size=11),
                                    height=30),

                        cells=dict(values=values,
                                    format=[None, ',.2f'],
                                   line_color='darkslategray',
                                   fill=dict(color=['paleturquoise', 'white']),
                                    font_size=10,
                                    height=20

                                    ))
                             ])
        fig.show()
    
    #8. Variance
    def table_var(self, df): 
        list_val=[]
        list_name=[]
        for i in range(0, len(df)):
            list_name.append("Var"+ str(i))
            values = df[i]['dyn_vcov']
            values = values.tolist()
            values = [values[i][i] for i in range(0, len(values))]
#             values.insert(0, df[i]['dyn_vars'].tolist())
            list_val.append(values)
        list_val.insert(0, df[0]['dyn_vars'].tolist())    
#         print(list_val)
        table = [go.Table(
                        columnwidth = [10,9],
                        header=dict(values=['Variable']+list_name,
                                   line_color='darkslategray',
                                    fill_color='royalblue',
                                    font=dict(color='white', size=11),
                                    height=30),

                        cells=dict(values=list_val,
                                    format=[None, ',.2f'],
                                   line_color='darkslategray',
                                   fill=dict(color=['paleturquoise', 'white']),
                                    font_size=10,
                                    height=20

                                    ))
                             ]
        layout = go.Layout(width=400, height=450, autosize=False, showlegend=False)
        
        fig = go.Figure(data=table, layout=layout)
        fig.show() 
#         return values
